<a href="https://colab.research.google.com/github/UranMai/Binding_affinity/blob/master/SPlitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries


In [81]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c rdkit rdkit pytorch

--2020-01-20 12:23:15--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.2.0=py37_0
    - ca-certificates==2019.10.16=0
    - certifi==2019.9.11=py37_0
    - cffi==1.13.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - conda==4.7.12=py37_0
    - cryptography==2.8=py37h1ba5d50_0
    - idna==2.8=py37_0
    - libedit==3.1.20181209=hc058e9b_0
    -

In [0]:
import sys
import os

from argparse import ArgumentParser
from itertools import product
from typing import List
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.Scaffolds import MurckoScaffold
from tqdm import tqdm

import math
import numpy as np
from collections import defaultdict
import logging
import random
from typing import Dict, List, Set, Tuple, Union

In [0]:
import pandas as pd
data = pd.read_csv('compound_1000.csv')

In [86]:
data['mol'] = data['smi'].apply(lambda x: Chem.MolFromSmiles(x))
data = data.dropna()
data = data.reset_index(drop=True)
data.head()


RDKit ERROR: [12:24:34] SMILES Parse Error: syntax error while parsing: none
RDKit ERROR: [12:24:34] SMILES Parse Error: Failed parsing SMILES 'none' for input: 'none'
RDKit ERROR: [12:24:34] SMILES Parse Error: syntax error while parsing: none
RDKit ERROR: [12:24:34] SMILES Parse Error: Failed parsing SMILES 'none' for input: 'none'


,Unnamed: 0,smi,Score,Score_bin,mol
0,0,NC(=O)c1ccc(-c2ccc3ccccc3n2)cc1,-30.421166,1,<rdkit.Chem.rdchem.Mol object at 0x7fef44b55f30>
1,1,Nc1nc2nc(-c3ccccc3)cc(-c3ccccc3)n2n1,-20.839885,0,<rdkit.Chem.rdchem.Mol object at 0x7fef44b55ee0>
2,2,O=C1[C@@H](N2CCc3cccc(F)c32)CCN1CC(F)(F)F,-30.615744,1,<rdkit.Chem.rdchem.Mol object at 0x7fef44b55f80>
3,3,CSCc1cc(C(=O)N2CCc3[nH]ncc3C2)ccc1F,-25.646399,0,<rdkit.Chem.rdchem.Mol object at 0x7fef44b1bad0>
4,4,CC(=O)Nc1ccc(NC(=O)CSc2ccc(F)cc2)cc1,-29.502308,0,<rdkit.Chem.rdchem.Mol object at 0x7fef44b1bf30>


https://github.com/swansonk14/chemprop/blob/master/chemprop/data

In [0]:
#Compute Bemis_Murcko scaffolds that contain ring systems
def generate_scaffold(mol):
    mol = Chem.MolFromSmiles(mol)
    scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=False)
    return scaffold

In [0]:
#compute total scaffolds and match which of them have same scaffolds
def scaffold_to_smiles(mols, use_indices: bool = False):
    scaffolds = defaultdict(set)
    for i, mol in tqdm(enumerate(mols), total=len(mols)):
        scaffold = generate_scaffold(mol)
        if use_indices:
            scaffolds[scaffold].add(i)
        else:
            scaffolds[scaffold].add(mol)

    return scaffolds

In [0]:
def scaffold_split(data,
                   sizes: Tuple[float, float, float] = (0.8, 0.1, 0.1),
                   balanced: bool = False,
                   seed: int = 0):
  

    assert sum(sizes) == 1

    # Split
    train_size, val_size, test_size = sizes[0] * len(data), sizes[1] * len(data), sizes[2] * len(data)
    train, val, test = [], [], []
    train_scaffold_count, val_scaffold_count, test_scaffold_count = 0, 0, 0

    # Map from scaffold to index in the data
    scaffold_to_indices = scaffold_to_smiles(data.smi, use_indices=True)

    index_sets = sorted(list(scaffold_to_indices.values()),
                        key=lambda index_set: len(index_set),
                        reverse=True)
      
    for index_set in index_sets:
        if len(train) + len(index_set) <= train_size:
            train += index_set
            train_scaffold_count += 1
        elif len(val) + len(index_set) <= val_size:
            val += index_set
            val_scaffold_count += 1
        else:
            test += index_set
            test_scaffold_count += 1

    
       
    

    # Map from indices to data
    train = [data.smi[i] for i in train]
    val = [data.smi[i] for i in val]
    test = [data.smi[i] for i in test]

    return train,val,test

In [150]:
train

array(['CSc1cccc(NS(=O)(=O)c2cc(C(=O)O)ccc2F)c1',
       'O=C(O)c1cc(F)ccc1NS(=O)(=O)c1ccc(F)c(F)c1F',
       'COC(=O)c1cccc(S(=O)(=O)Nc2ccccc2[C@H](O)C(F)(F)F)c1',
       'Cc1ccc(S(=O)(=O)Nc2ccc(OC(C)C)cc2)cc1C(=O)O',
       'COc1ccc(S(=O)(=O)N(C)c2cc(Cl)ccc2Cl)cc1',
       'COc1ccc(S(=O)(=O)N(C)c2ccc(OC)c(Cl)c2)cc1F',
       'CC[C@H](C)NS(=O)(=O)c1cc(C(=O)O)ccc1NCc1ccccc1',
       'COc1ccc(NCc2cc(Cl)ccc2O)cc1OC',
       'CCOc1cccc(CNc2ccc(OC)c(OC)c2)c1O',
       'COC(=O)c1ccc(CN(C)c2ccccc2[N+](=O)[O-])cc1',
       'CCNC(=O)COc1cccc(NC(=O)c2ccc(Br)s2)c1',
       'CCc1ccc(C(=O)Nc2ccc(OC)c(C(=O)N(C)C)c2)s1',
       'CCN(C)S(=O)(=O)c1csc(C(=O)Nc2ccccc2Cl)c1',
       'COCCN(C)S(=O)(=O)c1csc(C(=O)Nc2ccc(C)c(C)c2)c1',
       'COc1ccc(OC)c([C@@H](C)NC(=O)c2cccc(F)c2)c1',
       'Cc1ccc(C(=O)NCc2ccc(C(=O)O)cc2)cc1S(N)(=O)=O',
       'CCN(Cc1ccccc1[N+](=O)[O-])C(=O)c1cccc(N(C)C)c1',
       'CCCN(C)C(=O)Nc1cccc(S(=O)(=O)N2CCCCC2)c1',
       'CCCNC(=O)[C@H]1CCCN(S(=O)(=O)c2ccc(OCC)cc2)C1',
     

# test

In [121]:
train = scaffold_split(data)[0]
train = np.array(train)
val = scaffold_split(data)[1]
val = np.array(val)
test = scaffold_split(data)[2]
test = np.array(test)

100%|██████████| 998/998 [00:00<00:00, 2237.88it/s]


In [0]:
table_train = pd.DataFrame(train, columns=['smi'])
table_val = pd.DataFrame(val)
table_test = pd.DataFrame(test)

tables = [table_train, table_val, table_test]

In [0]:
table = pd.concat(tables)
table

In [0]:
table_train

In [134]:
len(list(set(data.smi).intersection(table_train.smi)))

798

In [0]:
for col in data.smi:
  for col1 in table_train.smi:
    if col == col1:
      

In [0]:
b = [data['smi'] == col for col in table_train.smi]
table_train['col'] = data.loc[b]
table_train